In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Implement me
# Set working directory
abspath = '/content/drive/My Drive/Colab Notebooks/NLP/NLP-Final-Project/NLP Project'

Mounted at /content/drive


In [2]:
import os
os.chdir(abspath)
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/NLP/NLP-Final-Project/NLP Project'

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

import dateutil.parser

In [2]:
# dictionary of text dates and links (March 2017 - June 2017)

# Febuary 12, 2017
url1 = 'https://www.reuters.com/article/us-airlines-united-pilot-idUSKBN15R0RC'

# MARCH 7, 2017
url2 = 'https://www.reuters.com/article/us-united-airlines-idUSKBN16E2P2'

#MARCH 26, 2017
url3 = 'https://www.reuters.com/article/us-unitedairlines-leggings-idUSKBN16X13G'

# April 7, 2017
url4 = 'https://www.reuters.com/article/us-usair-united-idUSTRE6365PN20100408'

# April 11, 2017
url5 = 'https://www.reuters.com/article/us-ual-passenger-shares-idUSKBN17D1L7'

# April 14, 2017
url6 = 'https://www.reuters.com/article/us-ual-scorpion-idUSKBN17G1EQ'

# APRIL 17, 2017
url7 = 'https://www.reuters.com/article/us-ual-passenger-couple-idUSKBN17J05Q'

# April 27, 2017
url8 = 'https://www.reuters.com/article/us-ual-passenger-idUSKBN17T2WM'

# May 2, 2017
url9 = 'https://www.reuters.com/article/us-ual-passenger-ceo-idUSKBN17Y1J8'

#MAY 4, 2017
url10 = 'https://www.reuters.com/article/ual-passenger-idUSL1N1I51WE'


#MAY 30, 2017
url11 = 'https://www.reuters.com/article/us-united-fine-idUSKBN18Q2EQ'

# June 3, 2017
url12 = 'https://www.reuters.com/article/us-venezuela-airlines-united-idUSKBN18U0TL'



url_dict = {'2017-02-12':url1,
            '2017-03-07':url2,
            '2017-03-26':url3,
            '2017-04-07':url4,
            '2017-04-11':url5,
            '2017-04-14':url6,
            '2017-04-17':url7,
            '2017-04-27':url8,
            '2017-05-02':url9,
            '2017-05-04':url10,
            '2017-05-30':url11,
            '2017-06-03':url12,
           }


In [3]:
# extract info from each website and store it in dataframe
df_text = pd.DataFrame(columns = ['Date','Header','Text'])
header = []
text = []
date = []
for key in url_dict:
    page = requests.get(url_dict[key]).text
    soup = BeautifulSoup(page, 'html.parser')
    date.append(datetime.strptime(key, '%Y-%m-%d').date())
    url_header = soup.find('h1').text
    header.append(url_header)
    url_text = soup.find('div', class_ ='ArticleBodyWrapper')
    url_text = url_text.text
    text.append(url_text)
df_text['Date']=date
df_text['Header']=header
df_text['Text']=text

# remove the author and "min read" sections
df_text_temp = [] 
for text in df_text['Text']:
    loc = text.find('(Reuters)')
    df_text_temp.append(text[loc+9:])
df_text['Text'] = df_text_temp




In [4]:
df_text['Date'] = df_text['Date'].apply(lambda x: pd.to_datetime(x))
df_text.dtypes

Date      datetime64[ns]
Header            object
Text              object
dtype: object

In [5]:
# import stock data of netflix
df_stock = pd.read_csv('UAL.csv', header=0)

df_stock['Date']=df_stock['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


df_stock = df_stock.drop(['High','Low','Close','Adj Close'], axis=1)
df_stock.dtypes

Date      datetime64[ns]
Open             float64
Volume             int64
dtype: object

In [6]:
# Merge two dataframes
df = pd.merge(df_text, df_stock, how='inner', on='Date')
df = df.sort_values('Date')

In [7]:
# show % change in stock price after news article publish
pct_stock = df['Open'].pct_change()*100
df['% Change in Stock Price'] = pct_stock
pct_volume = df['Volume'].pct_change()*100
df['% Change in Volume'] = pct_volume
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume
0,2017-03-07,"United looking at second-hand aircraft, rules ...",- United Continental Holdings UAKL.N is inter...,72.690002,3764000,NaN,NaN
1,2017-04-07,UAL pilots attack U.S. Airways merger prospect,- United Airlines' pilots union lashed out ag...,69.970001,2471100,-3.741919,-34.349097
2,2017-04-11,United Airlines faces mounting pressure over h...,- United Airlines UAL.N and its chief executi...,70.150002,17696500,0.257255,616.138562
3,2017-04-17,United Air removes couple traveling to wedding...,- An engaged couple were removed from a Unite...,69.510002,4761900,-0.912331,-73.091289
4,2017-04-27,United Airlines reaches settlement with passen...,- United Airlines UAL.N and the passenger who...,69.769997,5512200,0.374040,15.756316


# Processing the text


## import spacy libraries
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [8]:
# import spacy libraries
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [9]:

text_nlp = nlp(df['Text'][1]) 
# Remove stop words
text_nlp_clean = [word for word in text_nlp if word.is_stop == False]
# Lemmatize words
text_nlp_clean = [word.lemma_ for word in text_nlp_clean]

In [27]:
text_clean = []
for ind, row in df.iterrows():
    
    # lower casing
    text_lower = row['Text'].lower()
    
    # tokenization
    text_nlp = nlp(row['Text']) 
    
    # remove punctuation and empty space
    text_nlp_clean = [word.lower_ for word in text_nlp]
    
    # remove punctuation and empty space
    text_nlp_clean = [word for word in text_nlp if not word.is_punct | word.is_space]
    
    # Remove stop words
    text_nlp_clean = [word for word in text_nlp_clean if word.is_stop == False]
    
    # Lemmatize tokens
    text_nlp_clean = [word.lemma_ for word in text_nlp_clean]
    
    # add preprocessed text to list
    text_clean.append(text_nlp_clean)

# turn list of words to string
def to_string(list1):
    sentence = ""
    
    
    # traverse in the string   
    for word in list1:  
        sentence += str(word) 
    
    # return string   
    return sentence

# turn list of strings to string
to_string = []
for text in text_clean:
    to_string.append(' '.join(text))


df['Text (clean)'] = to_string
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume,Text (clean)
0,2017-03-07,"United looking at second-hand aircraft, rules ...",- United Continental Holdings UAKL.N is inter...,72.690002,3764000,NaN,NaN,United Continental Holdings UAKL.N interested ...
1,2017-04-07,UAL pilots attack U.S. Airways merger prospect,- United Airlines' pilots union lashed out ag...,69.970001,2471100,-3.741919,-34.349097,United Airlines pilot union lash effort merge ...
2,2017-04-11,United Airlines faces mounting pressure over h...,- United Airlines UAL.N and its chief executi...,70.150002,17696500,0.257255,616.138562,United Airlines UAL.N chief executive face mou...
3,2017-04-17,United Air removes couple traveling to wedding...,- An engaged couple were removed from a Unite...,69.510002,4761900,-0.912331,-73.091289,engage couple remove United Airlines flight Co...
4,2017-04-27,United Airlines reaches settlement with passen...,- United Airlines UAL.N and the passenger who...,69.769997,5512200,0.374040,15.756316,United Airlines UAL.N passenger drag Chicago f...


# Using a Pretrained Model 

In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [23]:
# a function to return sentiment score
def polarity_score(text):
    sid = SentimentIntensityAnalyzer()
    return sid.polarity_scores(text)['compound']

vader_score = []
for text in df['Text (clean)']:
    score = polarity_score(text)
    vader_score.append(score)

df['Vader Sentiment Score'] = vader_score

In [24]:
df.head()

Date  ... Vader Sentiment Score
0 2017-03-07  ...  0.9485              
1 2017-04-07  ...  0.9981              
2 2017-04-11  ...  0.9892              
3 2017-04-17  ...  0.9638              
4 2017-04-27  ...  0.9136              

[5 rows x 9 columns]

## Sentiment Analysis

In [76]:
# run this cell to import nltk
import nltk
from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 
from utils import process_tweet, build_freqs

In [77]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [78]:

# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [79]:
# split the data into two pieces (80-20), one for training and one for testing (validation set)  
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [80]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [81]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [82]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11346


In [83]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        
        x[0,1] += freqs.get((word,1),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [84]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

In [85]:
Y.shape

(8000, 1)

In [86]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import Pipeline

### SVM Parameters

In [87]:
param_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1]}  

In [88]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [89]:

# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet(tweet, freqs):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = classifier.predict(x)
    
    ### END CODE HERE ###
    
    return y_pred

In [90]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet_prob(tweet, freqs):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = classifier.predict_proba(x)
    
    ### END CODE HERE ###
    
    return y_pred

In [91]:

my_tweet = 'The plot was terrible and I was sad until the ending!'
predict_tweet(my_tweet, freqs)

array([0.])

In [92]:
sentiment = []
sentiment_prob = []
for i in df.iloc[:,2]:
    sentiment.append(predict_tweet(i, freqs)[0])
    sentiment_prob.append(predict_tweet_prob(i, freqs)[0][1])

In [93]:
df['sentiment'] = sentiment
df['sentiment_prob'] = sentiment_prob
df

Date  ... sentiment_prob
0 2017-03-07  ...  0.693922     
1 2017-04-07  ...  0.000996     
2 2017-04-11  ...  0.016666     
3 2017-04-17  ...  0.100628     
4 2017-04-27  ...  0.003092     
5 2017-05-02  ...  0.000056     
6 2017-05-04  ...  0.038566     
7 2017-05-30  ...  0.555865     

[8 rows x 15 columns]

# START OF DEEP LEARNING

In [16]:
import sklearn
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import sklearn
import sklearn.ensemble
from sklearn.metrics import accuracy_score, confusion_matrix
import sklearn.metrics
import tensorflow as tf
from scipy.special import softmax
import os
import numpy as np
import pandas as pd
import json
import torch
import torch.nn as nn
import nltk
from tqdm import tqdm
import spacy
import torch
import torchtext
import torchtext.data
import torch.nn as nn
import torch.nn.functional as F
from scipy.special import softmax
from torchtext.vocab import Vocab
from nltk.corpus import twitter_samples 

nlp = spacy.load('en')
nltk.download('punkt')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [18]:
# %% ----------------------------------- Hyper Parameters --------------------------------------------------------------
class Args:
    def __init__(self):
        self.seq_len = "get_max_from_data"
        self.embedding_dim = 50
        self.n_epochs = 10
        self.lr = 1e-2
        self.batch_size = 512
        self.train = True
        self.save_model = True


args = Args()

In [63]:
# %% ----------------------------------- Helper Functions --------------------------------------------------------------
def acc(x, y, return_labels=False):
    with torch.no_grad():
        logits = torch.empty(len(x), 2)
        for batch in range(len(x) // args.batch_size + 1):
            inds = slice(batch * args.batch_size, (batch + 1) * args.batch_size)
            logits[inds] = model(x[inds])
        pred_labels = np.argmax(logits.cpu().numpy(), axis=1)
    if return_labels:
        return pred_labels
    else:
        return 100*accuracy_score(y.cpu().numpy(), pred_labels)


def extract_vocab_dict_and_msl(sentences_train, sentences_dev):
    """ Tokenizes all the sentences and gets a dictionary of unique tokens and also the maximum sequence length """
    tokens, ms_len = [], 0
    for sentence in list(sentences_train) + list(sentences_dev):
        tokens_in_sentence = nltk.word_tokenize(sentence)
        if ms_len < len(tokens_in_sentence):
            ms_len = len(tokens_in_sentence)
        tokens += tokens_in_sentence
    token_vocab = {key: i for key, i in zip(set(tokens), range(1, len(set(tokens))+1))}
    if len(np.unique(list(token_vocab.values()))) != len(token_vocab):
        "There are some rep words..."
    return token_vocab, ms_len


def convert_to_ids(raw_sentences, vocab_dict, pad_to):
    """ Takes an NumPy array of raw text sentences and converts to a sequence of token ids """
    x = np.empty((len(raw_sentences), pad_to))
    for idx, sentence in enumerate(raw_sentences):
        word_ids = []
        for token in nltk.word_tokenize(sentence):

            if token not in vocab_dict.keys():
              continue 
            try:
                word_ids.append(vocab_dict[token])
            except:
                word_ids.append(vocab_dict[token])
        if pad_to < len(word_ids):
            x[idx] = word_ids[:pad_to]
        else:
            x[idx] = word_ids + [0] * (pad_to - len(word_ids))
    return x


def get_glove_embeddings(vocab_dict):
    with open("glove.6B.50d.txt", "r") as s:
        glove = s.read()
    embeddings_dict = {}
    for line in glove.split("\n")[:-1]:
        text = line.split()
        if text[0] in vocab_dict:
            embeddings_dict[vocab_dict[text[0]]] = torch.from_numpy(np.array(text[1:], dtype="float32"))
    return embeddings_dict


def get_glove_table(vocab_dict, glove_dict):
    lookup_table = torch.empty((len(vocab_dict)+2, 50))
    for token_id in sorted(vocab_dict.values()):
        if token_id in glove_dict:
            lookup_table[token_id] = glove_dict[token_id]
        else:
            lookup_table[token_id] = torch.zeros((1, 50))  # For unknown tokens
    lookup_table[0] = torch.zeros((1, 50))
    return lookup_table



In [20]:
data_train = pd.read_csv("original_data/train.tsv", sep="\t")
x_train_raw, y_train = data_train["sentence"].values, torch.LongTensor(data_train["label"].values).to(device)
data_dev = pd.read_csv("original_data/dev.tsv", sep="\t")
x_dev_raw, y_dev = data_dev["sentence"].values, torch.LongTensor(data_dev["label"].values).to(device)


In [25]:
# %% -------------------------------------- Data Prep ------------------------------------------------------------------
try:
    with open("example_prep_data/vocab_dict.json", "r") as s:
        token_ids = json.load(s)
    msl = np.load("example_prep_data/max_sequence_length.npy").item()
except:
    print("Tokenizing all the examples to get a vocab dict and the maximum sequence length...")
    token_ids, msl = extract_vocab_dict_and_msl(x_train_raw, x_dev_raw)
    os.mkdir("example_prep_data")
    with open("example_prep_data/vocab_dict.json", "w") as s:
        json.dump(token_ids, s)
    np.save("example_prep_data/max_sequence_length.npy", np.array([msl]))
if args.seq_len == "get_max_from_data":
    args.seq_len = msl

glove_embeddings = get_glove_embeddings(token_ids)

try:
    x_train = np.load("example_prep_data/prep_train_len{}.npy".format(args.seq_len))
    x_dev = np.load("example_prep_data/prep_dev_len{}.npy".format(args.seq_len))
except:
    print("Converting all the sentences to sequences of token ids...")
    x_train = convert_to_ids(x_train_raw, token_ids, args.seq_len)
    np.save("example_prep_data/prep_train_len{}.npy".format(args.seq_len), x_train)
    x_dev = convert_to_ids(x_dev_raw, token_ids, args.seq_len)
    np.save("example_prep_data/prep_dev_len{}.npy".format(args.seq_len), x_dev)




Converting all the sentences to sequences of token ids...


In [26]:
x_train, x_dev = torch.LongTensor(x_train).to(device), torch.LongTensor(x_dev).to(device)

In [27]:
# %% -------------------------------------- CNN Class ------------------------------------------------------------------
class CNN(nn.Module):
    def __init__(self, vocab_size):
        super(CNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size + 2, args.embedding_dim)

        self.conv1 = nn.Conv1d(args.embedding_dim, args.embedding_dim, 9)
        self.convnorm1 = nn.BatchNorm1d(args.embedding_dim)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(args.embedding_dim, args.embedding_dim, 9)
        self.convnorm2 = nn.BatchNorm1d(args.embedding_dim)
        self.pool2 = nn.MaxPool1d(2)

        self.conv3 = nn.Conv1d(args.embedding_dim, args.embedding_dim, 7)
        self.linear = nn.Linear(args.embedding_dim, 2)
        self.act = torch.relu

    def forward(self, x):
        # nn.Conv1d operates on the columns, each embedding dimension is considered as one channel
        x = self.embedding(x).permute(0, 2, 1)
        x = self.pool1(self.convnorm1(self.act(self.conv1(x))))
        x = self.pool2(self.convnorm2(self.act(self.conv2(x))))
        return self.linear(self.act(self.conv3(x)).reshape(-1, args.embedding_dim))



In [28]:
# %% -------------------------------------- Training Prep ----------------------------------------------------------
model = CNN(len(token_ids)).to(device)
look_up_table = get_glove_table(token_ids, glove_embeddings)
model.embedding.weight.data.copy_(look_up_table)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
criterion = nn.CrossEntropyLoss()

In [29]:
# %% -------------------------------------- Training Loop ----------------------------------------------------------
labels_ditrib = torch.unique(y_dev, return_counts=True)
print("The no information rate is {:.2f}".format(100*labels_ditrib[1].max().item()/len(y_dev)))
if args.train:
    acc_dev_best = 0
    print("Starting training loop...")
    for epoch in range(args.n_epochs):

        loss_train, train_steps = 0, 0
        model.train()
        total = len(x_train) // args.batch_size + 1  # Initiates a progress bar that will be updated for each batch
        with tqdm(total=total, desc="Epoch {}".format(epoch)) as pbar:  # "Epoch" will be updated for each epoch
            for batch in range(len(x_train)//args.batch_size + 1):
                inds = slice(batch*args.batch_size, (batch+1)*args.batch_size)
                optimizer.zero_grad()
                logits = model(x_train[inds])
                loss = criterion(logits, y_train[inds])
                loss.backward()
                optimizer.step()
                loss_train += loss.item()
                train_steps += 1
                pbar.update(1)  # Updates the progress and the training loss
                pbar.set_postfix_str("Training Loss: {:.5f}".format(loss_train / train_steps))

        model.eval()
        with torch.no_grad():
            y_dev_pred = model(x_dev)
            loss = criterion(y_dev_pred, y_dev)
            loss_test = loss.item()

        acc_dev = acc(x_dev, y_dev)
        print("Epoch {} | Train Loss {:.5f}, Train Acc {:.2f} - Test Loss {:.5f}, Test Acc {:.2f}".format(
            epoch, loss_train/train_steps, acc(x_train, y_train), loss_test, acc_dev))

        if acc_dev > acc_dev_best and args.save_model:
            torch.save(model.state_dict(), "cnn_sentiment.pt")
            print("The model has been saved!")
            acc_dev_best = acc_dev

Epoch 0:   0%|          | 0/132 [00:00<?, ?it/s]

The no information rate is 50.92
Starting training loop...


Epoch 1:   0%|          | 0/132 [00:00<?, ?it/s]

Epoch 0 | Train Loss 0.33875, Train Acc 94.27 - Test Loss 0.50666, Test Acc 80.50
The model has been saved!


Epoch 2:   0%|          | 0/132 [00:00<?, ?it/s]

...

Epoch 7 | Train Loss 0.03735, Train Acc 99.05 - Test Loss 0.88590, Test Acc 80.05


Epoch 9:   0%|          | 0/132 [00:00<?, ?it/s]

Epoch 8 | Train Loss 0.03333, Train Acc 98.72 - Test Loss 1.04044, Test Acc 80.85


Epoch 9: 100%|██████████| 132/132 [00:35<00:00,  3.73it/s, Training Loss: 0.02484]


Epoch 9 | Train Loss 0.02484, Train Acc 99.44 - Test Loss 0.97284, Test Acc 80.62


# Test of Deep Learning

In [30]:
x = None
for i in range(10):
    test = convert_to_ids([x_train_raw[i]], token_ids, args.seq_len)
    test = torch.LongTensor(test).to(device)
    pred = model(test).cpu().detach().numpy()[0]
    x = softmax(pred)
    print(softmax(pred),y_train[i])

[1.0000000e+00 2.5123225e-07] tensor(0)
[1.000000e+00 7.204974e-11] tensor(0)
[6.590116e-14 1.000000e+00] tensor(1)
[9.9999809e-01 1.7947648e-06] tensor(0)
[1.000000e+00 2.441419e-07] tensor(0)
[9.9998617e-01 1.3672282e-05] tensor(0)
[5.602653e-07 9.999995e-01] tensor(1)
[1.0325499e-05 9.9998951e-01] tensor(1)
[1.00000e+00 3.12389e-17] tensor(0)
[1.8741416e-08 1.0000000e+00] tensor(1)


In [31]:
df

Date  ... Vader Sentiment Score
0 2017-03-07  ...  0.9485              
1 2017-04-07  ...  0.9981              
2 2017-04-11  ...  0.9892              
3 2017-04-17  ...  0.9638              
4 2017-04-27  ...  0.9136              
5 2017-05-02  ...  0.9460              
6 2017-05-04  ...  0.9921              
7 2017-05-30  ...  0.8625              

[8 rows x 9 columns]

In [69]:

x = None
deep_learning_sentiment_prob = []

for i in df.iloc[:,7]:


  test = convert_to_ids([i.lower()], token_ids, args.seq_len)


  test = torch.LongTensor(test).to(device)
  pred = model(test).cpu().detach().numpy()[0]
  print(pred)

  x = softmax(pred)
  deep_learning_sentiment_prob.append(x[1])

[-2.786009   2.6891813]
[-0.02735396  0.07592878]
[ 3.3044693 -0.590564 ]
[-18.035051  12.57733 ]
[-0.02735396  0.07592878]
[-1.401726   2.2558806]
[-24.036062  13.958266]
[ 8.114444  -2.5411978]


In [70]:
deep_learning_sentiment_prob

[0.99582803,
 0.5257978,
 0.019937122,
 1.0,
 0.5257978,
 0.9748544,
 1.0,
 2.3566947e-05]

In [71]:
df['deep_learning_sentiment_prob'] = deep_learning_sentiment_prob

In [72]:
df

Date  ... deep_learning_sentiment_prob
0 2017-03-07  ...  0.995828                   
1 2017-04-07  ...  0.525798                   
2 2017-04-11  ...  0.019937                   
3 2017-04-17  ...  1.000000                   
4 2017-04-27  ...  0.525798                   
5 2017-05-02  ...  0.974854                   
6 2017-05-04  ...  1.000000                   
7 2017-05-30  ...  0.000024                   

[8 rows x 10 columns]

# Summary

In [73]:
# Converting % stock price change to % stock price change per day
days = [0]
for i in range(len(df['Date'])):
    if i != 0:
        date_diff = df['Date'][i]-df['Date'][i-1]
        day = date_diff.days
        days.append(day)
print(days)
df['Days Elapsed'] = days
df.head()
df['% Change in Stock Price Per Day'] = df['% Change in Stock Price']/df['Days Elapsed']
df.head()

[0, 31, 4, 6, 10, 5, 2, 26]


Date  ... % Change in Stock Price Per Day
0 2017-03-07  ... NaN                            
1 2017-04-07  ... -0.120707                      
2 2017-04-11  ...  0.064314                      
3 2017-04-17  ... -0.152055                      
4 2017-04-27  ...  0.037404                      

[5 rows x 12 columns]

In [96]:
df['Vader Sentiment'] = df['Vader Sentiment Score'].apply(lambda x: "positive" if x>0 else "negative")
df['Logistic Regression Sentiment'] = df['sentiment_prob'].apply(lambda x: "positive" if x>0.5 else "negative")
df['Stock Price Change'] = df['% Change in Stock Price Per Day'].apply(lambda x: "positive" if x>0 else "negative")
df['deep_learning_sentiment_prob'] = df['deep_learning_sentiment_prob'].apply(lambda x: "positive" if x>0.5 else "negative")

In [98]:
columns = ['Date','Header','Stock Price Change', 'Vader Sentiment', 'Logistic Regression Sentiment','deep_learning_sentiment_prob']
df_summary = df[columns]
df_summary.head()

,Date,Header,Stock Price Change,Vader Sentiment,Logistic Regression Sentiment,deep_learning_sentiment_prob
0,2017-03-07,"United looking at second-hand aircraft, rules out A380",negative,positive,positive,positive
1,2017-04-07,UAL pilots attack U.S. Airways merger prospect,negative,positive,negative,positive
2,2017-04-11,United Airlines faces mounting pressure over hospitalized passenger,positive,positive,negative,negative
3,2017-04-17,United Air removes couple traveling to wedding from plane,negative,positive,negative,positive
4,2017-04-27,United Airlines reaches settlement with passenger dragged from plane,positive,positive,negative,positive
